# Case Study 5 - Explore drivers of yield
## Description 
As a crop scientist, I want to integrate a mixture of environmental variables (soils, pests, etc.) with plant phenotyping data, so that I can interpret the relative importance of these aspects on plant yield.
## Case Breakdown 
- **Actors:** Crop scientist
- **Golas:** Data analysis between environmental data nad phenotype data
- **Scope:** Regional, plot-based
## Generalised case
I want to combine a suite of spatial variables at different scales across multiple sites so I can analyse the factors correlated with a variable of interest. (Subcase of Case Study 1)
## Comparable cases
- I want to aggregate iMapPests data for the same pest across multiple sites and locations so I can analyse the relationship between population levels and environmental context at the time and over the previous month, including weather (temperature, rainfall, humidity - all xyt), lunar phase (t) and greenness (xyt - see https://portal.tern.org.au/metadata/TERN/8542d90e-6e20-4ad8-b30d-0a171b61d3f5).
## Stakeholders 
- **Name:** TBD
- **Contact:** TBD@adelaide.edu.au


## Data Sources
Explore the USyd LLaraCampey data. Use the following as the suite of environmental layers for assessing:

1. ECa raster
2. Radiometric raster
3. A relevant rainfall measure - if we have (or can guess) the growing season dates or the yield measurement data for each plot, see if we can get the total rainfall over the period (or say 3 months up to yield)
4. A relevant low-temperature measure - number of days in the same period below e.g. 5C (not sure what the lowest temperatures would be)
5. A relevant high-temperature measure - as above.
6. A couple of kriged soil estimates (what ones are available?)

This should be for each year for which data are sufficient. Use the yield data as the actual values against which to assess importance. If we have any variety data, we could process separately for different varieties.

## Data Transformation
Describe the steps take to process the data. 

In [4]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE

## DataCube Generation
Describe the steps take to process the data. 

In [5]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE

## Data Analysis/Visulisation
Describe the steps take to process the data. 

In [6]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE